# Cloud COGs to Image Collection

## Setup

In [6]:
import ee
ee_project = 'akveg-map'

ee.Authenticate(auth_mode='notebook')  #  or !earthengine authenticate --auth_mode=gcloud

ee.Initialize(project=ee_project)
# ee.Authenticate()  #  or !earthengine authenticate --auth_mode=gcloud
# session = AuthorizedSession(ee.data.get_persistent_credentials())

In [8]:
import pandas as pd
from pathlib import Path
from datetime import datetime, timezone
import os
import re

from google.auth.transport.requests import AuthorizedSession

session = AuthorizedSession(
    ee.data.get_persistent_credentials().with_quota_project(ee_project)
)

## Dynamic World Counts, raw by MGRS tile

### Cleanup existing imageCollection, if needed

imageCollections cannot be deleted until all images inside them are deleted. For a cloud-backed image collection with hundreds or thousands of tiles, this can take a while

bash earthengine CLI
```
for i in `earthengine ls projects/akveg-map/assets/dynamic_world_metrics/s2_dw_monthly_counts_mgrs_v20250414b`; do earthengine rm $i; done

# earthengine rm projects/akveg-map/assets/s2_2019_2023_gMedian_v20240311
```

### Create empty image collections as target
TODO Automate creation of empty image collection.

For now, manually create empty image collection with earthengine CLI

```
earthengine set_project akveg-map
earthengine create collection projects/akveg-map/assets/dynamic_world_metrics/s2_dw_monthly_counts_mgrs_v20250414b
```

### Create list of cloud geotiffs in bucket and image collection

Create list of cogs in a bucket folder to load into an imageCollection.

Create list of cogs that have already been loaded into the imageCollection.

Run in bash in a conda env with gsutil and earthengine command line installed and authenticated.

TODO: Configure it to run directly in python

```
cd /data/gis/raster_base/Alaska/AKVegMap/dynamic_world

gsutil ls gs://akveg-data/s2_dw_v1_metrics/s2_dw_monthly_counts_mgrs_v20250414b/*.tif > dw_monthly_cogs.txt
earthengine ls projects/akveg-map/assets/dynamic_world_metrics/s2_dw_monthly_counts_mgrs_v20250414b > dw_monthly_cogs_inColl.txt

```

In [43]:
# Function to read a text file and return a list of paths
def read_paths_from_file(filepath):
    with open(filepath, 'r') as file:
        paths = file.read().splitlines()
    return paths

# Function to extract the last part of the path and remove the extension
def get_filename_without_extension(path):
    return os.path.splitext(os.path.basename(path))[0]


# Function to save a list of paths to a text file
def save_paths_to_file(paths, filepath):
    with open(filepath, 'w') as file:
        for path in paths:
            file.write(f"{path}\n")

def in_list_1_not_list_2(csv1_path, csv2_path):
    cogList = read_paths_from_file(csv1_path)
    cogListIC = read_paths_from_file(csv2_path)
    
    filenames1 = [get_filename_without_extension(path) for path in cogList]
    filenames2 = [get_filename_without_extension(path) for path in cogListIC]
    
    # Convert lists to pd Series
    filenames_series1 = pd.Series(filenames1, index=cogList)
    filenames_series2 = pd.Series(filenames2, index=cogListIC)
    
    # Find filenames in list1 that are not in list2
    unique_to_list1 = filenames_series1[~filenames_series1.isin(filenames2)]
    
    print("\nPaths in list1 that are not in list2 based on filenames:")
    print(unique_to_list1.index.tolist()[:5])
    print(len(unique_to_list1.index.tolist()))
    
    return unique_to_list1

In [ ]:
# Apply the functions to for full list and already loaded list of vhr_toa_cogs
unique_to_list1 = in_list_1_not_list_2('/data/gis/raster_base/Alaska/AKVegMap/dynamic_world/dw_monthly_cogs.txt',
                                       '/data/gis/raster_base/Alaska/AKVegMap/dynamic_world/dw_monthly_cogs_inColl.txt')

save_paths_to_file(unique_to_list1.index.tolist(), '/data/gis/raster_base/Alaska/AKVegMap/dynamic_world/dw_monthly_cogs_not_inColl.txt')


Paths in list1 that are not in list2 based on filenames:
['gs://akveg-data/s2_dw_v1_metrics/s2_dw_monthly_counts_mgrs_v20250414b/s2_dw_counts_mgrs_month05_03UUV_v20250414.tif', 'gs://akveg-data/s2_dw_v1_metrics/s2_dw_monthly_counts_mgrs_v20250414b/s2_dw_counts_mgrs_month06_01UCS_v20250414.tif']
2


### Open list of geotiffs to ingest

In [44]:
dw_cogs = pd.read_csv('/data/gis/raster_base/Alaska/AKVegMap/dynamic_world/dw_monthly_cogs_not_inColl.txt', header=None,names=['tif'])
print(dw_cogs[0:2])
print(len(dw_cogs.index))


                                                 tif
0  gs://akveg-data/s2_dw_v1_metrics/s2_dw_monthly...
1  gs://akveg-data/s2_dw_v1_metrics/s2_dw_monthly...
2


### Setup parameters

In [45]:
import json
# from urllib.parse import urlparse
import os
from pprint import pprint

# Earth Engine enabled Cloud Project.
project_folder = 'akveg-map'
# collection = 'reflectance_vhr/ortho_toa_images'

### View list of cogs to ingest
Optional, can skip when list is long.

In [46]:
for cog in dw_cogs['tif']:
    print(cog)

gs://akveg-data/s2_dw_v1_metrics/s2_dw_monthly_counts_mgrs_v20250414b/s2_dw_counts_mgrs_month05_03UUV_v20250414.tif
gs://akveg-data/s2_dw_v1_metrics/s2_dw_monthly_counts_mgrs_v20250414b/s2_dw_counts_mgrs_month06_01UCS_v20250414.tif


### Function to load list of gcs cogs to GEE imageCollection
comment out pprint and most print except when troubleshooting

In [21]:
#TODO Fix date extraction from filename to work for different filename formats
def load_gcs_cogs_to_collection(cogs, project_folder, collection, month_part, tile_part):
    # Request body as a dictionary.
    for cog in cogs['tif']:
      fileOnly = os.path.split(cog)[1]
      # print(fileOnly)

      cogName = fileOnly[:-4]
      print(cogName)
      
      # parts = fileOnly.split('_')
      # Split on both underscore and period
      parts = re.split(r'[_\.]', fileOnly)
      print(parts)
      
      monthxx = f"{parts[month_part]}"[-2:]
      month = int(monthxx)
      print(month)

      tile = f"{parts[tile_part]}"
      print(tile)

      # dt = datetime.strptime(yyyymmdd_hhmmss, "%Y%m%d_%H%M%S").replace(tzinfo=timezone.utc)
      # dt = f"{dt:%Y-%m-%dT%H:%M:%SZ}"  # Format string for ISO 8601 + Z
      # print(dt)
        
      request = {
        'type': 'IMAGE',
        'gcs_location': {
          'uris': cog
        },
        'properties': {
        #   'source': 'https://code.earthengine.google.com/067b10ee56537817756a3177a9138aee',
            # 'yyyymmdd_hhmmss': yyyymmdd_hhmmss,
            'month': monthxx,
            'tile': tile,
        },
        # 'startTime': dt,#'2023-01-01T00:00:00.000000000Z',
        # 'endTime': dt,#'2024-01-01T00:00:00.000000000Z',
      }

      pprint(json.dumps(request))

      # A folder (or ImageCollection) name and the new asset name.
      asset_id = collection+'/'+cogName
      # print(project_folder)
      # print(asset_id)
        
      url = 'https://earthengine.googleapis.com/v1alpha/projects/{}/assets?assetId={}'
      print(url)
      print(url.format(project_folder, asset_id))

      response = session.post(
        url = url.format(project_folder, asset_id),
        data = json.dumps(request)
      )

      # pprint(json.loads(response.content))
    print('done')


### Run it

In [22]:
load_gcs_cogs_to_collection(dw_cogs, project_folder, 'dynamic_world_metrics/s2_dw_monthly_counts_mgrs_v20250414b', 4, 5)


s2_dw_counts_mgrs_month05_03UUV_v20250414
['s2', 'dw', 'counts', 'mgrs', 'month05', '03UUV', 'v20250414', 'tif']
5
03UUV
('{"type": "IMAGE", "gcs_location": {"uris": '
 '"gs://akveg-data/s2_dw_v1_metrics/s2_dw_monthly_counts_mgrs_v20250414b/s2_dw_counts_mgrs_month05_03UUV_v20250414.tif"}, '
 '"properties": {"month": "05", "tile": "03UUV"}}')
https://earthengine.googleapis.com/v1alpha/projects/{}/assets?assetId={}
https://earthengine.googleapis.com/v1alpha/projects/akveg-map/assets?assetId=dynamic_world_metrics/s2_dw_monthly_counts_mgrs_v20250414b/s2_dw_counts_mgrs_month05_03UUV_v20250414
s2_dw_counts_mgrs_month06_01UCS_v20250414
['s2', 'dw', 'counts', 'mgrs', 'month06', '01UCS', 'v20250414', 'tif']
6
01UCS
('{"type": "IMAGE", "gcs_location": {"uris": '
 '"gs://akveg-data/s2_dw_v1_metrics/s2_dw_monthly_counts_mgrs_v20250414b/s2_dw_counts_mgrs_month06_01UCS_v20250414.tif"}, '
 '"properties": {"month": "06", "tile": "01UCS"}}')
https://earthengine.googleapis.com/v1alpha/projects/{}/assets

In [12]:
import ee
ee.Initialize()

operations = ee.data.listOperations()

# Filter for failed or cancelled tasks
failed_ops = [
    op for op in operations
    if op.get('metadata', {}).get('state') in ['FAILED', 'CANCELLED']
]

# Print summary
for op in failed_ops:
    meta = op['metadata']
    print(f"Task ID: {op['name'].split('/')[-1]}")
    print(f"Description: {meta.get('description', 'N/A')}")
    print(f"State: {meta.get('state')}")
    print(f"Error: {meta.get('errorMessage', 'None')}")
    print('-' * 40)

# Need to redo two (maybe no granules for that month?)
# s2_dw_counts_month05_03UUV_v20250414
# s2_dw_counts_month06_01UCS_v20250414

Task ID: GXLT5QBVB64VCFROC5U47ZMV
Description: s2_dw_counts_month05_03UUV_v20250414
State: FAILED
Error: None
----------------------------------------
Task ID: 273PJSOXO4FQ7GXVGDPDSR52
Description: s2_dw_counts_month06_01UCS_v20250414
State: FAILED
Error: None
----------------------------------------
Task ID: 32BQ2LQ46S6JIDHWVN7Q7NYS
Description: s2_dw_counts_month07_06VUQ_v20250414
State: FAILED
Error: None
----------------------------------------
Task ID: 4GSBFOYQF6U34VUT7EZU4EOY
Description: s2_dw_counts_month07_06VUQ_v20250414
State: FAILED
Error: None
----------------------------------------
Task ID: 6FIFZTCXEMGQGGHGDOLJUWAI
Description: s2_dw_counts_month07_06VUQ_v20250414
State: CANCELLED
Error: None
----------------------------------------
Task ID: UNSCUHRMP243PK23WQ3D2BWX
Description: s2_dw_counts_month07_06VUQ_v20250414
State: FAILED
Error: None
----------------------------------------


## Dynamic World Percentages (May-Sept) by AKALB 50km tile

### Cleanup existing imageCollection, if needed

imageCollections cannot be deleted until all images inside them are deleted. For a cloud-backed image collection with hundreds or thousands of tiles, this can take a while

bash earthengine CLI
```
for i in `earthengine ls projects/akveg-map/assets/dynamic_world_metrics/s2_dw_monthly_counts_mgrs_v20250414b`; do earthengine rm $i; done

# earthengine rm projects/akveg-map/assets/s2_2019_2023_gMedian_v20240311
```

### new code for cleaning existing imageCollection

bash earthengine CLI
```
COLLECTION="projects/akveg-map/assets/dynamic_world_metrics/s2_dw_percentages_56789_v20250414"
earthengine ls $COLLECTION | xargs -P 20 -I {} earthengine rm {}
```

### Create empty image collections as target
TODO Automate creation of empty image collection.

For now, manually create empty image collection with earthengine CLI

```
earthengine set_project akveg-map
earthengine create collection projects/akveg-map/assets/dynamic_world_metrics/s2_dw_percentages_56789_v20250414b
```

### Create list of cloud geotiffs in bucket and image collection

Create list of cogs in a bucket folder to load into an imageCollection.

Create list of cogs that have already been loaded into the imageCollection.

Run in bash in a conda env with gsutil and earthengine command line installed and authenticated.

TODO: Configure it to run directly in python

```
cd /data/gis/raster_base/Alaska/AKVegMap/dynamic_world

gsutil ls gs://akveg-data/s2_dw_v1_metrics/s2_dw_pct_akalb_050_v20250414/*.tif > dw_percentage_cogs.txt
earthengine ls projects/akveg-map/assets/dynamic_world_metrics/s2_dw_percentages_56789_v20250414 > dw_percentage_cogs_inColl.txt

```

In [5]:
# Function to read a text file and return a list of paths
def read_paths_from_file(filepath):
    with open(filepath, 'r') as file:
        paths = file.read().splitlines()
    return paths

# Function to extract the last part of the path and remove the extension
def get_filename_without_extension(path):
    return os.path.splitext(os.path.basename(path))[0]


# Function to save a list of paths to a text file
def save_paths_to_file(paths, filepath):
    with open(filepath, 'w') as file:
        for path in paths:
            file.write(f"{path}\n")

def in_list_1_not_list_2(csv1_path, csv2_path):
    cogList = read_paths_from_file(csv1_path)
    cogListIC = read_paths_from_file(csv2_path)
    
    filenames1 = [get_filename_without_extension(path) for path in cogList]
    filenames2 = [get_filename_without_extension(path) for path in cogListIC]
    
    # Convert lists to pd Series
    filenames_series1 = pd.Series(filenames1, index=cogList)
    filenames_series2 = pd.Series(filenames2, index=cogListIC)
    
    # Find filenames in list1 that are not in list2
    unique_to_list1 = filenames_series1[~filenames_series1.isin(filenames2)]
    
    print("\nPaths in list1 that are not in list2 based on filenames:")
    print(unique_to_list1.index.tolist()[:5])
    print(len(unique_to_list1.index.tolist()))
    
    return unique_to_list1

In [6]:
# Apply the functions to for full list and already loaded list of vhr_toa_cogs
unique_to_list1 = in_list_1_not_list_2('/data/gis/raster_base/Alaska/AKVegMap/dynamic_world/dw_percentage_cogs.txt',
                                       '/data/gis/raster_base/Alaska/AKVegMap/dynamic_world/dw_percentage_cogs_inColl.txt')

save_paths_to_file(unique_to_list1.index.tolist(), '/data/gis/raster_base/Alaska/AKVegMap/dynamic_world/dw_percentage_cogs_not_inColl.txt')


Paths in list1 that are not in list2 based on filenames:
['gs://akveg-data/s2_dw_v1_metrics/s2_dw_pct_akalb_050_v20250414/s2_dw_pct_56789_AK050H01V31_v20250414.tif', 'gs://akveg-data/s2_dw_v1_metrics/s2_dw_pct_akalb_050_v20250414/s2_dw_pct_56789_AK050H01V32_v20250414.tif', 'gs://akveg-data/s2_dw_v1_metrics/s2_dw_pct_akalb_050_v20250414/s2_dw_pct_56789_AK050H01V33_v20250414.tif', 'gs://akveg-data/s2_dw_v1_metrics/s2_dw_pct_akalb_050_v20250414/s2_dw_pct_56789_AK050H02V31_v20250414.tif', 'gs://akveg-data/s2_dw_v1_metrics/s2_dw_pct_akalb_050_v20250414/s2_dw_pct_56789_AK050H02V32_v20250414.tif']
279


### Open list of geotiffs to ingest

In [7]:
dw_cogs = pd.read_csv('/data/gis/raster_base/Alaska/AKVegMap/dynamic_world/dw_percentage_cogs_not_inColl.txt', header=None,names=['tif'])
print(dw_cogs[0:2])
print(len(dw_cogs.index))


                                                 tif
0  gs://akveg-data/s2_dw_v1_metrics/s2_dw_pct_aka...
1  gs://akveg-data/s2_dw_v1_metrics/s2_dw_pct_aka...
279


### Setup parameters

In [8]:
import json
# from urllib.parse import urlparse
import os
from pprint import pprint

# Earth Engine enabled Cloud Project.
project_folder = 'akveg-map'
# collection = 'reflectance_vhr/ortho_toa_images'

### View list of cogs to ingest
Optional, can skip when list is long.

In [9]:
for cog in dw_cogs['tif']:
    print(cog)

gs://akveg-data/s2_dw_v1_metrics/s2_dw_pct_akalb_050_v20250414/s2_dw_pct_56789_AK050H01V31_v20250414.tif
gs://akveg-data/s2_dw_v1_metrics/s2_dw_pct_akalb_050_v20250414/s2_dw_pct_56789_AK050H01V32_v20250414.tif
gs://akveg-data/s2_dw_v1_metrics/s2_dw_pct_akalb_050_v20250414/s2_dw_pct_56789_AK050H01V33_v20250414.tif
gs://akveg-data/s2_dw_v1_metrics/s2_dw_pct_akalb_050_v20250414/s2_dw_pct_56789_AK050H02V31_v20250414.tif
gs://akveg-data/s2_dw_v1_metrics/s2_dw_pct_akalb_050_v20250414/s2_dw_pct_56789_AK050H02V32_v20250414.tif
gs://akveg-data/s2_dw_v1_metrics/s2_dw_pct_akalb_050_v20250414/s2_dw_pct_56789_AK050H02V33_v20250414.tif
gs://akveg-data/s2_dw_v1_metrics/s2_dw_pct_akalb_050_v20250414/s2_dw_pct_56789_AK050H03V32_v20250414.tif
gs://akveg-data/s2_dw_v1_metrics/s2_dw_pct_akalb_050_v20250414/s2_dw_pct_56789_AK050H03V33_v20250414.tif
gs://akveg-data/s2_dw_v1_metrics/s2_dw_pct_akalb_050_v20250414/s2_dw_pct_56789_AK050H04V34_v20250414.tif
gs://akveg-data/s2_dw_v1_metrics/s2_dw_pct_akalb_050_v2

### Function to load list of gcs cogs to GEE imageCollection
comment out pprint and most print except when troubleshooting

In [10]:
#TODO Fix date extraction from filename to work for different filename formats
def load_gcs_cogs_to_collection(cogs, project_folder, collection, month_part, tile_part, version_counts_part):
    # Request body as a dictionary.
    for cog in cogs['tif']:
      fileOnly = os.path.split(cog)[1]
      # print(fileOnly)

      cogName = fileOnly[:-4]
      print(cogName)
      
      # parts = fileOnly.split('_')
      # Split on both underscore and period
      parts = re.split(r'[_\.]', fileOnly)
      print(parts)
      
      months = parts[month_part]
      # month = int(monthxx)
      print(months)

      tile = f"{parts[tile_part]}"
      print(tile)

      version_counts = f"{parts[version_counts_part]}"
      print(version_counts)

      # dt = datetime.strptime(yyyymmdd_hhmmss, "%Y%m%d_%H%M%S").replace(tzinfo=timezone.utc)
      # dt = f"{dt:%Y-%m-%dT%H:%M:%SZ}"  # Format string for ISO 8601 + Z
      # print(dt)
        
      request = {
        'type': 'IMAGE',
        'gcs_location': {
          'uris': cog
        },
        'properties': {
        #   'source': 'https://code.earthengine.google.com/067b10ee56537817756a3177a9138aee',
            # 'yyyymmdd_hhmmss': yyyymmdd_hhmmss,
            'months': months,
            'tile': tile,
            'version_counts': version_counts
        },
        # 'startTime': dt,#'2023-01-01T00:00:00.000000000Z',
        # 'endTime': dt,#'2024-01-01T00:00:00.000000000Z',
      }

      pprint(json.dumps(request))

      # A folder (or ImageCollection) name and the new asset name.
      asset_id = collection+'/'+cogName
      # print(project_folder)
      # print(asset_id)
        
      url = 'https://earthengine.googleapis.com/v1alpha/projects/{}/assets?assetId={}'
      print(url)
      print(url.format(project_folder, asset_id))

      response = session.post(
        url = url.format(project_folder, asset_id),
        data = json.dumps(request)
      )

      # pprint(json.loads(response.content))
    print('done')


### Run it

In [11]:
load_gcs_cogs_to_collection(dw_cogs, project_folder, 'dynamic_world_metrics/s2_dw_percentages_56789_v20250414', 3, 4, 5)


s2_dw_pct_56789_AK050H01V31_v20250414
['s2', 'dw', 'pct', '56789', 'AK050H01V31', 'v20250414', 'tif']
56789
AK050H01V31
v20250414
('{"type": "IMAGE", "gcs_location": {"uris": '
 '"gs://akveg-data/s2_dw_v1_metrics/s2_dw_pct_akalb_050_v20250414/s2_dw_pct_56789_AK050H01V31_v20250414.tif"}, '
 '"properties": {"months": "56789", "tile": "AK050H01V31", "version_counts": '
 '"v20250414"}}')
https://earthengine.googleapis.com/v1alpha/projects/{}/assets?assetId={}
https://earthengine.googleapis.com/v1alpha/projects/akveg-map/assets?assetId=dynamic_world_metrics/s2_dw_percentages_56789_v20250414/s2_dw_pct_56789_AK050H01V31_v20250414
s2_dw_pct_56789_AK050H01V32_v20250414
['s2', 'dw', 'pct', '56789', 'AK050H01V32', 'v20250414', 'tif']
56789
AK050H01V32
v20250414
('{"type": "IMAGE", "gcs_location": {"uris": '
 '"gs://akveg-data/s2_dw_v1_metrics/s2_dw_pct_akalb_050_v20250414/s2_dw_pct_56789_AK050H01V32_v20250414.tif"}, '
 '"properties": {"months": "56789", "tile": "AK050H01V32", "version_counts": '


In [ ]:
import ee
import pandas as pd
from datetime import datetime, timezone

# Initialize Earth Engine
ee.Initialize()

# Parse Alaska time in format: 'YYYY-MM-DD HH:MM:SS -0800'
def parse_alaska_offset(s):
    return datetime.strptime(s, '%Y-%m-%d %H:%M:%S %z').astimezone(timezone.utc)

# Input times (Alaska time with UTC offset)
start_str = '2026-02-03 21:20:00 -0800'
end_str   = '2026-02-05 12:00:00 -0800'

start_dt = parse_alaska_offset(start_str)
end_dt   = parse_alaska_offset(end_str)

# List tasks
ops = ee.data.listOperations()
print(ops)

# Filter and collect metadata
all_keys = set()
filtered_ops = []
for op in ops:
    meta = op.get('metadata', {})
    create_time = meta.get('createTime')
    if create_time:
        create_dt = datetime.fromisoformat(create_time.replace('Z', '+00:00'))
        if start_dt <= create_dt <= end_dt:
            filtered_ops.append(op)
            all_keys.update(meta.keys())

# Build rows
all_keys = sorted(all_keys)
rows = []
for op in filtered_ops:
    meta = op.get('metadata', {})
    row = {key: meta.get(key, None) for key in all_keys}
    row['task_id'] = op.get('name', '').split('/')[-1]
    row['state'] = 'DONE' if op.get('done') else 'RUNNING'
    rows.append(row)

# Save to CSV
df = pd.DataFrame(rows)
# df.to_csv('/data/gis/raster_base/Alaska/AKVegMap/dynamic_world/gee_tasks_all_metadata_20250424.csv', index=False)
# df.to_csv('./gee_tasks_all_metadata_20250424.csv', index=False)

total = df['batchEecuUsageSeconds'].sum()
cost = f"${total/3600*0.4:,.2f}"
print('EECUs:', total, cost)

[{'name': 'projects/akveg-map/operations/6NF4743ZSBQ56U2XUESMVIDI', 'metadata': {'@type': 'type.googleapis.com/google.earthengine.v1alpha.OperationMetadata', 'state': 'SUCCEEDED', 'description': 's2_dw_counts_56789_AK050H38V38_v20250414', 'priority': 100, 'createTime': '2026-02-04T18:36:55.215521Z', 'updateTime': '2026-02-05T07:46:41.729332Z', 'startTime': '2026-02-05T07:37:19.270752Z', 'endTime': '2026-02-05T07:46:41.729332Z', 'type': 'EXPORT_IMAGE', 'scriptUri': 'https://code.earthengine.google.com/9f48dff8029e67ce326b7a74ef84b451?noload=1', 'destinationUris': ['https://console.developers.google.com/storage/browser/akveg-data/s2_dw_v1_metrics/s2_dw_pct_akalb_050_v20250414/'], 'attempt': 1, 'progress': 1, 'stages': [{'displayName': 'Create Local Files', 'completeWorkUnits': 1, 'totalWorkUnits': '1', 'description': 'Computation and writing of temporary files.'}, {'displayName': 'Write Files to Destination', 'completeWorkUnits': 1, 'totalWorkUnits': '1', 'description': 'Uploading of fil

KeyError: 'batchEecuUsageSeconds'

In [17]:
import ee
ee.Initialize()

operations = ee.data.listOperations()

# Filter for failed or cancelled tasks
failed_ops = [
    op for op in operations
    if op.get('metadata', {}).get('state') in ['FAILED', 'CANCELLED']
]

# Print summary
for op in failed_ops:
    meta = op['metadata']
    print(f"Task ID: {op['name'].split('/')[-1]}")
    print(f"Description: {meta.get('description', 'N/A')}")
    print(f"State: {meta.get('state')}")
    print(f"Error: {meta.get('errorMessage', 'None')}")
    print('-' * 40)

# Need to redo two (maybe no granules for that month?)
# s2_dw_counts_month05_03UUV_v20250414
# s2_dw_counts_month06_01UCS_v20250414

Task ID: YYMPO3EHHR2GGDIAT7IR37JQ
Description: alaska_floodplain_training_data_30000
State: FAILED
Error: None
----------------------------------------
Task ID: LDUT7G55XQTZA7ICZN7PNA6Q
Description: alaska_floodplain_training_data_40000
State: FAILED
Error: None
----------------------------------------
Task ID: 7VNZY3N3DMC55LLDLLIVJ3O5
Description: alaska_floodplain_training_data_25000
State: FAILED
Error: None
----------------------------------------
Task ID: KGY5OUYEUEPPYJP3GG6S23H3
Description: alaska_floodplain_training_data_v5
State: FAILED
Error: None
----------------------------------------
Task ID: BCM4WG5TEI2I4I647NHJ7KLC
Description: alaska_floodplain_training_data_v3
State: FAILED
Error: None
----------------------------------------
Task ID: JZY4R347K6F4A2FAMP3E7AIG
Description: Alaska_Floodplain_Step1_Export_to_Asset_v4-dsw
State: CANCELLED
Error: None
----------------------------------------
